## Cilia segmentation
The purpose of this notebook is to segment cilia by training the APOC pixel classifier. Both a semantic segmentation to segment the cilia channel and binarise, and object segmentation to classify the cilia intro three types are used to complete the segmentation.

In [ ]:
from apoc import PixelClassifier
import matplotlib.pyplot as plt
import napari
import numpy as np
from readlif.reader import LifFile
from skimage.io import imsave
from skimage.exposure import rescale_intensity
from skimage.morphology import (
    disk,
    binary_opening,
    binary_closing,
    binary_dilation,
    binary_erosion,
    remove_small_objects,
    label,
    remove_small_holes,
)
from skimage.measure import label
from tifffile import TiffFile

In [ ]:
def lifloader(path, index):
    """
    This function loads the .lif file experiment by indicating the path where the file is found and the image index (position in the folder).
    TTiffFileunction reconstructs the image by using its channels and z stack. It returns an ndarray.
    Parameters:
    path: path
        Path where the .lif file is found.
    index: integer
        Indicates the image to be later opened from the experiment file (starting from 0).
    """
    file = LifFile(path)
    img = file.get_image(index)

    # The Z-stack images are piled up in their corresponding channels and the image is converted to an array.
    all_img = []
    for c in range(img.channels):
        this_z_stack = []
        for z in range(img.nz):
            this_z_stack.append(np.array(img.get_frame(z, 0, c, 0)))
        this_z_stack = rescale_intensity(np.asarray(this_z_stack))
        all_img.append(this_z_stack)
    all_img_array = np.array(all_img)

    # The image format is reshaped by changing the order of variables. This is done so Napari is able to read the image correctly.
    all_img_reshaped = np.reshape(
        all_img_array, (img.nz, img.channels, img.dims[0], img.dims[1])
    )
    all_img_reshaped2 = all_img_reshaped.transpose(1, 0, 2, 3)
    plt.imshow(all_img_reshaped2[0, 30, :, :])

    return all_img_reshaped2

In [ ]:
img = lifloader(
    "D:/estela/data/microscopy/leica_stellaris/20231025_p1_cd13_opn_arl13b/20231025_p1_cd13_opn_arl13b.lif",
    5,
)
cilia_channel = img[1]

viewer = napari.Viewer()
viewer.add_image(cilia_channel, scale=(0.3, 0.144, 0.144))
napari.utils.nbscreenshot(viewer)

In [ ]:
for_training_file = TiffFile(
    "D:/estela/results/20240304/cilia_for_training.tif"
)
for_training = for_training_file.asarray()

viewer = napari.Viewer()
viewer.add_image(cilia_channel, scale=(0.3, 0.144, 0.144))
viewer.add_labels(for_training, opacity=1)
napari.utils.nbscreenshot(viewer)

In [ ]:
segmenter = PixelClassifier(
    opencl_filename="D:/estela/results/20240304/PixelClassifier_2.cl"
)
prediction = segmenter.predict(cilia_channel)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(cilia_channel)  # , scale = (0.3, 0.144, 0.144))
viewer.add_labels(prediction.get(), opacity=0.2)
# viewer.add_labels(for_training, opacity=1)
napari.utils.nbscreenshot(viewer)

In [ ]:
eroded_prediction = binary_erosion((np.array(prediction) > 1))
polished_prediction = remove_small_objects(eroded_prediction, min_size=200)
dilated_prediction = binary_dilation(polished_prediction)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(cilia_channel)  # , scale = (0.3, 0.144, 0.144))
viewer.add_labels(prediction, opacity=0.2)
viewer.add_labels(eroded_prediction, opacity=0.2)
viewer.add_labels(polished_prediction, opacity=0.2)
viewer.add_labels(dilated_prediction, opacity=0.2)
# viewer.add_labels(for_training, opacity=1)
napari.utils.nbscreenshot(viewer)

In [ ]:
labelled_prediction = label(dilated_prediction)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(cilia_channel)  # , scale = (0.3, 0.144, 0.144))
viewer.add_labels(dilated_prediction, opacity=0.2)
viewer.add_labels(labelled_prediction)
# viewer.add_labels(for_training, opacity=1)
napari.utils.nbscreenshot(viewer)

In [ ]:
imsave("labelled_prediction.tif", labelled_prediction)